# 6. Realizando a Predição

In [1]:
#manipulação dos dados
import pandas as pd 

#configuração da pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', False)
pd.set_option('max_colwidth', None)

#Serialização do Modelo
import pickle

vamos agora pegar uma base de dados que não foi usada para treinar o modelo e fazer a predição, lembrando que essa base deve ter o mesmo formato da Tabela Analitica dos Dados usada para treinar o modelo

In [2]:
df = pd.read_csv('test_MiniCurso.csv', sep=',')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
0,88,technician,married,high.school,no,unknown,unknown,cellular,nov,mon,1591,22,447,8,success
1,46,services,unknown,illiterate,no,yes,no,cellular,jun,wed,1480,12,2,4,nonexistent
2,94,entrepreneur,divorced,university.degree,no,unknown,no,telephone,jun,wed,2608,18,999,0,success
3,54,student,unknown,basic.9y,yes,no,yes,cellular,nov,fri,875,11,999,1,success
4,58,student,unknown,high.school,yes,unknown,yes,telephone,sep,tue,1879,43,763,0,failure


In [3]:
df.tail()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
995,66,unknown,single,high.school,yes,yes,yes,telephone,apr,mon,1011,42,381,3,success
996,38,services,married,university.degree,yes,yes,yes,telephone,apr,thu,2615,35,84,3,success
997,39,unknown,unknown,basic.9y,yes,no,no,cellular,sep,thu,1198,50,600,0,failure
998,93,technician,single,basic.6y,no,no,yes,cellular,nov,wed,3578,21,999,2,success
999,36,admin.,single,basic.4y,unknown,yes,yes,cellular,feb,wed,1764,12,7,1,nonexistent


Vamos tratar os dados para chegar em uma tabela igual à ABT, veja que esses dados não tem a variável resposta

In [4]:
df['difficulty'] = -1 #desconhecido
#1 ligação e já converteu
df.loc[(df['poutcome'] == 'success') & (df['previous'].between(0,1)), 'difficulty'] = 0 #fácil
# 2 a 4 ligações e converteu
df.loc[(df['poutcome'] == 'success') & (df['previous'].between(2,4)), 'difficulty'] = 1 #médio
# 5 a 7 ligações e converteu
df.loc[(df['poutcome'] == 'success') & (df['previous'].between(5,7)), 'difficulty'] = 2 #difícil
# se não converteu
df.loc[(df['poutcome'] == 'failure'), 'difficulty'] = 3 #muito difícil
# se não teve contato anterior
df.loc[(df['poutcome'] == 'nonexistent'), 'difficulty'] = 4 #sem contato anterior

expl_cat = df[['job', 'marital', 'education','default', 'housing','loan', 'contact','month', 'day_of_week','poutcome', 'difficulty']]
expl_num = df[['age','duration','campaign','pdays','previous']]

expl_cat_encoding = pd.get_dummies(expl_cat, prefix_sep='_', columns=expl_cat.columns, drop_first=True, dtype='int')

df_abt = expl_num.merge(expl_cat_encoding, left_index=True, right_index=True)

expl_num_feature_selection = expl_num[['duration', 'campaign', 'pdays']]
expl_cat_feature_selection = expl_cat[['month', 'poutcome', 'difficulty']]
expl_cat_feature_selection = pd.get_dummies(expl_cat_feature_selection, prefix_sep='_', 
                                            columns=expl_cat_feature_selection.columns, 
                                            drop_first=True, dtype = int)

explicativas_modelagem = expl_num_feature_selection.merge(expl_cat_feature_selection, left_index=True, right_index=True)
explicativas_modelagem.drop(columns=['difficulty_0', 'month_feb', 'month_jan'], inplace=True)
explicativas_modelagem.head()




,duration,campaign,pdays,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_nonexistent,poutcome_success,difficulty_1,difficulty_2,difficulty_3,difficulty_4
0,1591,22,447,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,1480,12,2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
2,2608,18,999,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
3,875,11,999,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,1879,43,763,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


## 6.1 Carregando o Modelo Treinado


In [5]:
modelo_pickle = pickle.load(open('ModeloGridMiniCurso.pkl', 'rb'))
modelo_pickle

GridSearchCV(cv=2, estimator=GradientBoostingClassifier(),
             param_grid={'min_samples_leaf': [2, 100],
                         'min_samples_split': [2, 40],
                         'n_estimators': [100, 200, 500],
                         'random_state': [42]},
             scoring=make_scorer(roc_auc_score, response_method='predict'))

## 6.2 Prevendo os valores

In [6]:
previsao = modelo_pickle.predict(explicativas_modelagem)
pd.DataFrame(previsao).head(20)

,0
0,1
1,1
2,1
3,0
4,1
5,0
6,1
7,1
8,1
9,0


In [7]:
df_prevista = df.merge(pd.DataFrame(previsao), left_index=True, right_index=True)
df_prevista.rename(columns={0:'y'}, inplace=True)
df_prevista['y'] = df_prevista['y'].replace({1: 'yes', 0: 'no'})
df_prevista.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,difficulty,y
0,88,technician,married,high.school,no,unknown,unknown,cellular,nov,mon,1591,22,447,8,success,-1,yes
1,46,services,unknown,illiterate,no,yes,no,cellular,jun,wed,1480,12,2,4,nonexistent,4,yes
2,94,entrepreneur,divorced,university.degree,no,unknown,no,telephone,jun,wed,2608,18,999,0,success,0,yes
3,54,student,unknown,basic.9y,yes,no,yes,cellular,nov,fri,875,11,999,1,success,0,no
4,58,student,unknown,high.school,yes,unknown,yes,telephone,sep,tue,1879,43,763,0,failure,3,yes
5,49,unemployed,single,high.school,no,yes,yes,telephone,oct,wed,3298,43,319,1,nonexistent,4,no
6,41,services,single,unknown,yes,no,yes,cellular,jun,mon,1287,3,612,2,nonexistent,4,yes
7,67,housemaid,unknown,professional.course,no,unknown,no,telephone,sep,tue,2826,13,187,9,failure,3,yes
8,21,management,divorced,basic.9y,unknown,unknown,yes,cellular,dec,fri,953,39,745,7,failure,3,yes
9,79,self-employed,married,basic.9y,yes,unknown,no,cellular,feb,fri,2627,20,786,1,success,0,no


# 7. Outras ideias

- Será que dá prever a idade das pessoas baseadas nos parâmetros ?
Sim, usando modelos regressores

- Podemos também separar as pessoas em grupos! Usando modelos de clusterização !!

Mas devido ao tempo vamos deixar isso para outra oportunidade!!!